In [ ]:
# Show plots in Jupyter notebooks
%matplotlib inline

# Reload modules whenever they change
# (for development purposes)
%load_ext autoreload
%autoreload 2

# Make clusterking package available even without installation
import sys
sys.path = ["../../"] + sys.path

In [ ]:
import clusterking as ck
from clusterking.stability.subsamplestability import SubSampleStabilityTester
from clusterking.cluster import HierarchyCluster
import numpy as np
import pandas as pd
import tqdm

In [ ]:
d = ck.Data("output/tutorial_basics.sql")

In [ ]:
ssst = SubSampleStabilityTester()
ssst.set_basic_config(fraction=0.999, repeat=1000)

In [ ]:
c = HierarchyCluster()
c.set_max_d(0.2)

In [ ]:
r = ssst.run(d, c)

In [ ]:
r.df.hist(["fom", "nclusters"], density=True, layout=(1, 2), figsize=(15, 5));

In [ ]:
r.df.describe()

In [ ]:
r.df["fom"].mean()

In [ ]:
x = []
fom = []
nclusters = []
ssst = SubSampleStabilityTester()
ssst.set_progress_bar(False)
pts = list(np.linspace(0.7, 0.999, 20)) + list(np.linspace(0.9, 0.999, 19))
for fract in tqdm.tqdm(pts):
    x.append(fract)
    ssst.set_basic_config(fraction=fract, repeat=200)
    r = ssst.run(d, c)
    fom.append(r.df["fom"].mean())
    nclusters.append(r.df["nclusters"].mean())

In [ ]:
df = pd.DataFrame()
df["fract"] = x
df["fom"] = fom
df["nclusters"] = nclusters

In [ ]:
df.plot.scatter("fract", "fom")